# Задание 5.1 - Word2Vec

В этом задании мы натренируем свои word vectors на очень небольшом датасете.
Мы будем использовать самую простую версию word2vec, без negative sampling и других оптимизаций.

Перед запуском нужно запустить скрипт `download_data.sh` чтобы скачать данные.

Датасет и модель очень небольшие, поэтому это задание можно выполнить и без GPU.

In [1]:
#@title
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#@title
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt

# We'll use Principal Component Analysis (PCA) to visualize word vectors,
# so make sure you install dependencies from requirements.txt!
from sklearn.decomposition import PCA 

%matplotlib inline
device = torch.device("cuda:0")

In [3]:
#@title
import os
import numpy as np

class StanfordTreeBank:
    '''
    Wrapper for accessing Stanford Tree Bank Dataset
    https://nlp.stanford.edu/sentiment/treebank.html
    
    Parses dataset, gives each token and index and provides lookups
    from string token to index and back
    
    Allows to generate random context with sampling strategy described in
    word2vec paper:
    https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf
    '''
    def __init__(self):
        self.index_by_token = {}
        self.token_by_index = []

        self.sentences = []

        self.token_freq = {}
        
        self.token_reject_by_index = None

    def load_dataset(self, folder):
        filename = os.path.join(folder, "datasetSentences.txt")

        with open(filename, "r", encoding="latin1") as f:
            l = f.readline() # skip the first line
            
            for l in f:
                splitted_line = l.strip().split()
                words = [w.lower() for w in splitted_line[1:]] # First one is a number
                    
                self.sentences.append(words)
                for word in words:
                    if word in self.token_freq:
                        self.token_freq[word] +=1 
                    else:
                        index = len(self.token_by_index)
                        self.token_freq[word] = 1
                        self.index_by_token[word] = index
                        self.token_by_index.append(word)
        self.compute_token_prob()
                        
    def compute_token_prob(self):
        words_count = np.array([self.token_freq[token] for token in self.token_by_index])
        words_freq = words_count / np.sum(words_count)
        
        # Following sampling strategy from word2vec paper:
        # https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf
        self.token_reject_by_index = 1- np.sqrt(1e-5/words_freq)
    
    def check_reject(self, word):
        return np.random.rand() > self.token_reject_by_index[self.index_by_token[word]]
        
    def get_random_context(self, context_length=5):
        """
        Returns tuple of center word and list of context words
        """
        sentence_sampled = []
        while len(sentence_sampled) <= 2:
            sentence_index = np.random.randint(len(self.sentences)) 
            sentence = self.sentences[sentence_index]
            sentence_sampled = [word for word in sentence if self.check_reject(word)]
    
        center_word_index = np.random.randint(len(sentence_sampled))
        
        words_before = sentence_sampled[max(center_word_index - context_length//2,0):center_word_index]
        words_after = sentence_sampled[center_word_index+1: center_word_index+1+context_length//2]
        
        return sentence_sampled[center_word_index], words_before+words_after
    
    def num_tokens(self):
        return len(self.token_by_index)
        
data = StanfordTreeBank()
data.load_dataset("/content/drive/My Drive/Colab Notebooks/stanfordSentimentTreebank/stanfordSentimentTreebank/")

print("Num tokens:", data.num_tokens())
for i in range(5):
    center_word, other_words = data.get_random_context(5)
    print(center_word, other_words)

Num tokens: 19538
ironies ['schrader', 'relies']
quentin ['spawn', 'fools', 'handful', 'branched']
counter-cultural ["n't", 'definitive', '--', 'makers']
swim ['killer', 'whale', 'through']
diaz ['kept', 'cameron', 'prisoner', 'cage']


# Имплеменируем PyTorch-style Dataset для Word2Vec

Этот Dataset должен сгенерировать много случайных контекстов и превратить их в сэмплы для тренировки.

Напоминаем, что word2vec модель получает на вход One-hot вектор слова и тренирует простую сеть для предсказания на его основе соседних слов.
Из набора слово-контекст создается N сэмплов (где N - количество слов в контексте):

Например:

Слово: `orders` и контекст: `['love', 'nicest', 'to', '50-year']` создадут 4 сэмпла:
- input: `orders`, target: `love`
- input: `orders`, target: `nicest`
- input: `orders`, target: `to`
- input: `orders`, target: `50-year`

Все слова на входе и на выходе закодированы через one-hot encoding, с размером вектора равным количеству токенов.

In [4]:
#@title
class Word2VecPlain(Dataset):
    '''
    PyTorch Dataset for plain Word2Vec.
    Accepts StanfordTreebank as data and is able to generate dataset based on
    a number of random contexts
    '''
    def __init__(self, data, num_contexts=100):
        '''
        Initializes Word2VecPlain, but doesn't generate the samples yet
        (for that, use generate_dataset)
        Arguments:
        data - StanfordTreebank instace
        num_contexts - number of random contexts to use when generating a dataset
        '''
        # TODO: Implement what you need for other methods!
        self.data = data
        self.num_contexts = num_contexts
        self.input = []
        self.output = []
        self.num_tokens = self.data.num_tokens()
        
    
    def generate_dataset(self):
        '''
        Generates dataset samples from random contexts
        Note: there will be more samples than contexts because every context
        can generate more than one sample
        '''
        # TODO: Implement generating the dataset
        # You should sample num_contexts contexts from the data and turn them into samples
        # Note you will have several samples from one context
        for i in range(self.num_contexts):
            center_word, other_words = self.data.get_random_context()
            for word in other_words:
                self.input.append(center_word)
                self.output.append(word)
        
    def __len__(self):
        '''
        Returns total number of samples
        '''
        return len(self.input)
    
    def __getitem__(self, index):
        '''
        Returns i-th sample
        
        Return values:
        input_vector - torch.Tensor with one-hot representation of the input vector
        output_index - index of the target word (not torch.Tensor!)
        '''
        # TODO: Generate tuple of 2 return arguments for i-th sample 
        input_vector = torch.zeros([self.data.num_tokens()]).float()
        input_word = self.input[index]
        word_index_in_data = self.data.index_by_token[input_word]
        input_vector[word_index_in_data] = 1
        
        output_word = self.output[index]
        output_index = self.data.index_by_token[output_word]
        
        return input_vector, output_index
        

dataset = Word2VecPlain(data, 10)
dataset.generate_dataset()
input_vector, target = dataset[3]
print("Sample - input: %s, target: %s" % (input_vector, int(target))) # target should be able to convert to int
assert isinstance(input_vector, torch.Tensor)
assert torch.sum(input_vector) == 1.0
assert input_vector.shape[0] == data.num_tokens()

Sample - input: tensor([0., 0., 0.,  ..., 0., 0., 0.]), target: 10347


# Создаем модель и тренируем ее

In [6]:
#@title
# Create the usual PyTorch structures
dataset = Word2VecPlain(data, 30000)
dataset.generate_dataset()

# # We'll be training very small word vectors!
wordvec_dim = 10

# # We can use a standard sequential model for this
nn_model = nn.Sequential(
            nn.Linear(dataset.num_tokens, wordvec_dim, bias=False),
            nn.Linear(wordvec_dim, dataset.num_tokens, bias=False), 
         )
nn_model.type(torch.FloatTensor)

nn_model.to(device)


Sequential(
  (0): Linear(in_features=19538, out_features=10, bias=False)
  (1): Linear(in_features=10, out_features=19538, bias=False)
)

In [7]:
#@title
def extract_word_vectors(nn_model):
    '''
    Extracts word vectors from the model
    
    Returns:
    input_vectors: torch.Tensor with dimensions (num_tokens, num_dimensions)
    output_vectors: torch.Tensor with dimensions (num_tokens, num_dimensions)
    '''
    # TODO: Implement extracting word vectors from param weights
    # return tuple of input vectors and output vectos 
    # Hint: you can access weights as Tensors through nn.Linear class attributes
    input_vectors = torch.t([param.data for param in nn_model.parameters()][0].clone().detach())
    output_vectors = [param.data for param in nn_model.parameters()][1].clone().detach()
    return input_vectors, output_vectors
 


In [8]:
#@title
untrained_input_vectors, untrained_output_vectors = extract_word_vectors(nn_model)
assert untrained_input_vectors.shape == (data.num_tokens(), wordvec_dim)
assert untrained_output_vectors.shape == (data.num_tokens(), wordvec_dim)

In [9]:
#@title
def train_model(model, dataset, train_loader, optimizer, scheduler, num_epochs):
    '''
    Trains plain word2vec using cross-entropy loss and regenerating dataset every epoch
    
    Returns:
    loss_history, train_history
    '''
    
    loss = nn.CrossEntropyLoss().type(torch.FloatTensor)
    
    loss_history = []
    train_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        dataset.generate_dataset() # Regenerate dataset every epoch
         # TODO Implement training for this model
        # Note we don't have any validation set here because our purpose is the word vectors,
        # not the predictive performance of the model
        # And don't forget to step the learing rate scheduler!
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y_gpu.shape[0]
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        scheduler.step()      
        print("Epoch %i, Average loss: %f, Train accuracy: %f" % (epoch, ave_loss, train_accuracy))
        
    return loss_history, train_history

# Ну и наконец тренировка!

Добейтесь значения ошибки меньше **8.0**.

In [10]:
#@title
# Finally, let's train the model!

# TODO: We use placeholder values for hyperparameters - you will need to find better values!
optimizer = optim.SGD(nn_model.parameters(), lr=1.35, weight_decay=0)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=20)

loss_history, train_history = train_model(nn_model, dataset, train_loader, optimizer, scheduler, 15)

Epoch 0, Average loss: 9.876924, Train accuracy: 0.000906
Epoch 1, Average loss: 9.834677, Train accuracy: 0.004519
Epoch 2, Average loss: 9.579104, Train accuracy: 0.006007
Epoch 3, Average loss: 9.180169, Train accuracy: 0.007656
Epoch 4, Average loss: 8.822652, Train accuracy: 0.013554
Epoch 5, Average loss: 8.514903, Train accuracy: 0.020760
Epoch 6, Average loss: 8.285566, Train accuracy: 0.026582


KeyboardInterrupt: ignored

In [ ]:
#@title
dataset = Word2VecPlain(data, 30000)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=20)

In [ ]:
import torch.nn as nn
import torch.optim as optim

def approx_loss_reg_strength_learning_rate(trainer_func, train_loader, \
                                            loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor), num_epochs = 5, \
                                            num_experiments = 50, reg_strength_10_exp = (-6, -1), learning_rate_10_exp = (-1, 0.5)): 
    """
    ONLY FOR: Pytorch TENSORS, CUDA platform, optim.SGD  
    
    trainer_func - should return loss_history, train_accur_history, val_accur_history BY EPOCHS
    loss - for example nn.CrossEntropyLoss().type(torch.FloatTensor)
    nmb_experiments, int >= 100
    reg_strength_10_exp, tulip (-6, 0)   
    learning_rate_10_exp, tulip (-6, 0) 
    
    
    """
    loss_history = [] # history by hyperparams !!! 
    train_history = [] # history by hyperparams !!!
     
    reg_strength_history = []
    learning_rate_history = []

    for count in range(num_experiments):
        print('Count #: %d'% (count+1))
        reg_strength = 10**np.random.uniform(reg_strength_10_exp[0],reg_strength_10_exp[1]) 
        learning_rate = 10**np.random.uniform(learning_rate_10_exp[0],learning_rate_10_exp[1])
        print('reg_strength: %f'% (reg_strength))
        print('learning_rate: %f'% (learning_rate))
        wordvec_dim = 10

        nn_model = nn.Sequential(
            nn.Linear(dataset.num_tokens, wordvec_dim, bias=False),
            nn.Linear(wordvec_dim, dataset.num_tokens, bias=False), 
         )
        nn_model.type(torch.FloatTensor)
        nn_model.to(device)

        optimizer = optim.SGD(nn_model.parameters(), lr=learning_rate, weight_decay=reg_strength)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)
        loss_, train_ = trainer_func(nn_model, dataset, train_loader, optimizer, scheduler, num_epochs)
        loss_history.append(loss_[-1])
        train_history.append(train_[-1])
        reg_strength_history.append(reg_strength)
        learning_rate_history.append(learning_rate)
        
    approx_reg_strength = reg_strength_history[np.argmin(loss_history)]
    approx_learning_rate = learning_rate_history[np.argmin(loss_history)]
    approx_loss = np.min(loss_history)
    print('best loss achieved: %f at reg_strength %f and learning_rate  %f' % (approx_loss, approx_reg_strength, approx_learning_rate))
    return approx_loss, approx_reg_strength, approx_learning_rate 



In [ ]:
#@title
approx_loss, approx_reg_strength, approx_learning_rate = approx_loss_reg_strength_learning_rate(train_model, train_loader,  \
                                            loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor), num_epochs = 5, \
                                            num_experiments = 30, reg_strength_10_exp = (-6, -1), learning_rate_10_exp = (-1, 1))

In [ ]:
#@title
# Visualize training graphs
plt.subplot(211)
plt.plot(train_history)
plt.subplot(212)
plt.plot(loss_history) 

# Визуализируем вектора для разного вида слов до и после тренировки

В случае успешной тренировки вы должны увидеть как вектора слов разных типов (например, знаков препинания, предлогов и остальных) разделяются семантически.

Студенты - в качестве выполненного задания присылайте notebook с диаграммами!

In [12]:
trained_input_vectors, trained_output_vectors = extract_word_vectors(nn_model)
assert trained_input_vectors.shape == (data.num_tokens(), wordvec_dim)
assert trained_output_vectors.shape == (data.num_tokens(), wordvec_dim)


In [ ]:
def visualize_vectors(input_vectors, output_vectors, title=''):
    full_vectors = torch.cat((input_vectors, output_vectors), 0)
    full_vectors = full_vectors.cpu()
    wordvec_embedding = PCA(n_components=2).fit_transform(full_vectors)

    # Helpful words form CS244D example
    # http://cs224d.stanford.edu/assignment1/index.html
    visualize_words = {'green': ["the", "a", "an"], 
                      'blue': [",", ".", "?", "!", "``", "''", "--"], 
                      'brown': ["good", "great", "cool", "brilliant", "wonderful", 
                              "well", "amazing", "worth", "sweet", "enjoyable"],
                      'orange': ["boring", "bad", "waste", "dumb", "annoying", "stupid"],
                      'red': ['tell', 'told', 'said', 'say', 'says', 'tells', 'goes', 'go', 'went']
                     }

    plt.figure(figsize=(7,7))
    plt.suptitle(title)
    for color, words in visualize_words.items():
        points = np.array([wordvec_embedding[data.index_by_token[w]] for w in words])
        for i, word in enumerate(words):
            plt.text(points[i, 0], points[i, 1], word, color=color,horizontalalignment='center')
        plt.scatter(points[:, 0], points[:, 1], c=color, alpha=0.3, s=0.5)

visualize_vectors(untrained_input_vectors, untrained_output_vectors, "Untrained word vectors")
visualize_vectors(trained_input_vectors, trained_output_vectors, "Trained word vectors")